# Introduction

In this program, users can put the text they would like to analyze into the variable input_text, and the program would output whether this text is offensive language or hate speech or neither of them, along with the category of this text is it is offensive language or hate speech.

# Input Text

In [ ]:
# input text
# change to the input text needs to be analyzed
# this example is a twitter style text on sports with a short offensive language does not have any specific topic
input_text = "Las Vegas Aces win their first WNBA title! Chelsea Gray MVP First pro sports champ for Vegas! get to hit any bitch he wants to"

# Train Model<br>

In this part, we would use the best model architecture of the hate speech detection and topic classfication through the experiments we did. Since the performance has already been evaluated during previous experiments (see seperated file for each model experiments and their evaluation), we would use the whole dataset for training instead of train-test-split.

In [2]:
import numpy as np
import keras as kr
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.layers import SimpleRNN, Bidirectional, MaxPooling1D, GlobalMaxPooling1D, GRU

from wordcloud import WordCloud

from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
import transformers

In [ ]:
import pandas as pd
import numpy as np
import os
#visualizations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#consistent sized plot
from pylab import rcParams
rcParams['figure.figsize']=12,5
rcParams['axes.labelsize']=12
rcParams['xtick.labelsize']=12
rcParams['ytick.labelsize']=12

In [4]:
from transformers import BertTokenizer
import numpy as np
import time
import datetime
import random
import pandas as pd
import os, re
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
import tensorflow as tf
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup

In [ ]:
import warnings
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)
#text preprocessing libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
import re
pd.options.display.max_columns = None

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#variable to record the output from two models
hate_speech_classified_output = None
text_category_output = None

In [6]:
# train hate speech dection model
CLEAN_TWEETS = False
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

df = pd.read_csv('hate_speech_labeled_data.csv')

def strip_all_entities(x):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
if CLEAN_TWEETS == False:
    df['tweet']=df['tweet'].apply(strip_all_entities)
sentences = df['tweet'].values
labels = df['class'].values

train, Teal = train_test_split(df, random_state=1508, shuffle=True, test_size=0.2)
test, validation = train_test_split(Teal, random_state=1508, shuffle=True, test_size=0.5)

trn_sentences = train['tweet'].values
train_labels = train['class'].values

tst_sentences = test['tweet'].values
test_labels = test['class'].values

val_sentences = validation['tweet'].values
validation_labels = validation['class'].values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def bert_encode(data, max_len) :
    input_ids = []
    attention_masks = []


    for i in range(len(data)):
        encoded = tokenizer.encode_plus(data[i],
                                        add_special_tokens=True,
                                        max_length = max_len,
                                        padding = 'max_length',
                                        truncation = True,
                                        return_attention_mask=True)

        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return np.array(input_ids),np.array(attention_masks)
MAX_LEN = 64
train_inputs, train_masks = bert_encode(trn_sentences,MAX_LEN)
validation_inputs, validation_masks = bert_encode(val_sentences,MAX_LEN)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

optimizer = torch.optim.AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)

    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 5 ========
Training...
  Batch    40  of    620.    Elapsed: 0:00:15.
  Batch    80  of    620.    Elapsed: 0:00:28.
  Batch   120  of    620.    Elapsed: 0:00:42.
  Batch   160  of    620.    Elapsed: 0:00:56.
  Batch   200  of    620.    Elapsed: 0:01:10.
  Batch   240  of    620.    Elapsed: 0:01:25.
  Batch   280  of    620.    Elapsed: 0:01:39.
  Batch   320  of    620.    Elapsed: 0:01:53.
  Batch   360  of    620.    Elapsed: 0:02:07.
  Batch   400  of    620.    Elapsed: 0:02:21.
  Batch   440  of    620.    Elapsed: 0:02:35.
  Batch   480  of    620.    Elapsed: 0:02:49.
  Batch   520  of    620.    Elapsed: 0:03:03.
  Batch   560  of    620.    Elapsed: 0:03:17.
  Batch   600  of    620.    Elapsed: 0:03:32.

  Average training loss: 0.31
  Training epcoh took: 0:03:39

Running Validation...
  Accuracy: 0.91
  Validation took: 0:00:10

======== Epoch 2 / 5 ========
Training...
  Batch    40  of    620.    Elapsed: 0:00:14.
  Batch    80  of    620.    Elap

In [ ]:
bert_base_model = transformers.TFAutoModel.from_pretrained('bert-large-uncased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased')
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    print(df.head())
    return df
def tokenize_texts(text, tokenizer, max_len):
    text = text.tolist()
    tokenized_output = tokenizer(
        text = text,
        padding='max_length',
        max_length=max_len,
        truncation=True,
        return_tensors='np',
        is_split_into_words=False,
        stride = 0,
        return_overflowing_tokens=False,
    )
    tokenized_input_ids = tokenized_output['input_ids']
    return tokenized_input_ids
def split_data(df, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(df['article'], df['category'], test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test
def one_hot_encode(labels, num_classes=10):
    one_hot_encoded_label = tf.keras.utils.to_categorical(labels, num_classes=num_classes, dtype='int32')
    return one_hot_encoded_label
df = load_and_preprocess_data('data_preprocessed.csv')

X_train, X_test, y_train, y_test = split_data(df, test_size=0.2, random_state=42)

max_len = 60
tokenizer_x_train = tokenizer(
        text = X_train.astype('str').tolist(),
        padding='max_length',
        max_length=max_len,
        truncation=True,
        return_tensors='np',
        is_split_into_words=False,
        stride = 0,
        return_overflowing_tokens=False,
    )
X_train_tokenized = tokenizer_x_train['input_ids']

tokenizer_x_test = tokenizer(
        text = X_test.astype('str').tolist(),
        padding='max_length',
        max_length=max_len,
        truncation=True,
        return_tensors='np',
        is_split_into_words=False,
        stride = 0,
        return_overflowing_tokens=False,
    )
X_test_tokenized = tokenizer_x_test['input_ids']

encoder = LabelEncoder()
y_train = one_hot_encode(encoder.fit_transform(y_train))
y_test = one_hot_encode(encoder.transform(y_test))


def construct_bert_based_classifier(bert_layer, total_classes=10):

    bert_input = tf.keras.layers.Input(shape=(60,), dtype=tf.int32, name='bert_input_layer')
    dropout_seq = tf.keras.layers.Dropout(0.3, name='dropout_for_generalization')(bert_layer(bert_input)[0][:, 0, :])
    prediction_layer = tf.keras.layers.Dense(total_classes, activation='softmax', name='prediction_layer')(dropout_seq)
    bert_classifier = tf.keras.Model(inputs=bert_input, outputs=prediction_layer)
    bert_classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])

    return bert_classifier

text_classification_model = construct_bert_based_classifier(bert_base_model)

text_classification_model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

        category                                            article
0         COMEDY  of the funniest tweets about cats and dogs thi...
1      PARENTING  the funniest tweets from parents this week sep...
2         SPORTS  maury wills basestealing shortstop for dodgers...
3  ENTERTAINMENT  golden globes returning to nbc in january afte...
4       POLITICS  biden says us forces would defend taiwan if ch...
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 60)]              0         
                                                                 
 tf_bert_model_3 (TFBertMod  TFBaseModelOutputWithPo   335141888 
 el)                         olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 60, 1024),                       
                              pooler_output=(None

In [ ]:

BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE

train_data = tf.data.Dataset.from_tensor_slices((X_train_tokenized, y_train)).repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO)
test_data = tf.data.Dataset.from_tensor_slices((X_test_tokenized, y_test)).batch(BATCH_SIZE)

steps_per_epoch = 300
training_history = text_classification_model.fit(train_data, steps_per_epoch=steps_per_epoch, epochs=5)


Epoch 1/5
300/300 [==============================] - 369s 1s/step - loss: 0.6905 - accuracy: 0.7860
Epoch 2/5
300/300 [==============================] - 368s 1s/step - loss: 0.5857 - accuracy: 0.8172
Epoch 3/5
300/300 [==============================] - 368s 1s/step - loss: 0.5407 - accuracy: 0.8304
Epoch 4/5
300/300 [==============================] - 367s 1s/step - loss: 0.5290 - accuracy: 0.8306
Epoch 5/5
300/300 [==============================] - 368s 1s/step - loss: 0.5014 - accuracy: 0.8382


# Generate output

In [ ]:
# go through hate speech dection model
def hate_speech_preprocess(input_text):
  input_text = simplify(input_text)
  input_text.replace(r'@\w+','')
  input_text.replace(r'http\S+','')
  tokenizer = TweetTokenizer(preserve_case=True)
  input_text = tokenizer.tokenize(input_text)
  input_text = remove_stopwords(input_text)
  input_text = remove_hashsymbols(input_text)
  input_text = rem_shortwords(input_text)
  input_text = tokenizer.tokenize(input_text)
  input_text = rem_digits(input_text)
  input_text = tokenizer.tokenize(input_text)
  input_text = rem_nonalpha(input_text)
  input_text = ' '.join(input_text)
  input_text = vectorizer.transform([input_text])
  input_text_hate_speech = input_text
  return input_text_hate_speech
input_text_hate_speech = hate_speech_preprocess(input_text)
hate_speech_classified_output = hate_speech_classifier.predict(input_text_hate_speech)
if hate_speech_classified_output == 0:
    print('This is a hate speech')
elif hate_speech_classified_output == 1:
    print('This is an offensive language')
elif hate_speech_classified_output==2:
    print('This is not a hate speech or offensive language')
else:
    print("error in hate speech detection")

This is an offensive language


In [ ]:
# go through the text classification model
def text_classification_preprocess(input_text):
    input_text = simplify(input_text)
    max_len = 60
    tokenizer_x_train = tokenizer(
            text = input_text,
            padding='max_length',
            max_length=max_len,
            truncation=True,
            return_tensors='np',
            is_split_into_words=False,
            stride = 0,
            return_overflowing_tokens=False,
        )
    X_train_tokenized = tokenizer_x_train['input_ids']
    return X_train_tokenized

input_text_classification_preprocessed = text_classification_preprocess(input_text)
y_pred = text_classification_model.predict(input_text_classification_preprocessed)
predicted_category = encoder.classes_[np.argmax(y_pred, axis=1)]
text_category_output = predicted_category[0]

1/1 [==============================] - 0s 118ms/step


In [ ]:
if hate_speech_classified_output == 2:
  print('This is not a hate speech or offensive language')
elif hate_speech_classified_output == 0:
  print('This is a hate speech on ', text_category_output)
elif hate_speech_classified_output == 1:
  print('This is an offensive language on', text_category_output.lower())
else:
  print("error in hate speech detection")


This is an offensive language on sports
